In [ ]:
#!pip uninstall tensorflow

In [ ]:
#!pip install tensorflow-gpu==2.0.0

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers
print(tf.__version__)
import time
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
from sklearn.metrics import roc_auc_score
import random
import itertools
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from sklearn.metrics import confusion_matrix



2.3.0


In [2]:
size = 25

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

![title](擷取.PNG)

In [4]:
def gernate_data(n, nu, seed, range_, multi, size):
    ## normal
    np.random.seed(seed=seed)
    y_train = np.ones((int(n*(1-nu)),))
    mean = np.random.uniform(-5, 5, size)
    cov_  = np.random.uniform(0, 1, (size, size))* 3
    cov = np.dot(cov_.T,cov_)
    x_train = np.random.multivariate_normal(mean, cov, (int(n*(1-nu))))
    x_train = x_train.astype('float32')

    
    
    np.random.seed(seed=seed)
    x_trains_ = None ## normal + outlier
    for i in range(multi):
        mean = np.random.uniform(-range_, range_, size)
        cov_ = np.random.randn(size, size)
        cov = np.dot(cov_.T,cov_)
        x_train_ = np.random.multivariate_normal(mean, cov, (int(n*nu/multi)), check_valid  = 'raise')
        try:
            x_trains_ = np.vstack((x_trains_, x_train_))
        except:
            x_trains_ = x_train_
    ## 畫圖
    '''fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x_train[:, 0], x_train[:, 1],x_train[:, 2], s=0.9, label = 'normal')
    ax.scatter(x_trains_[:,0], x_trains_[:,1], x_trains_[:,2], s=0.9, label = 'outlier')
    plt.legend()
    plt.title('normal + outlier')
    plt.show()'''

    y_trains_ = np.zeros(((len(x_trains_),)))
    x_train_mix = np.vstack((x_trains_, x_train)).astype('float32')
    y_train_mix = np.hstack((y_trains_, y_train)).astype('float32')
    
    return x_train_mix, y_train_mix

## 參數設定

In [5]:
h_dim = [10, 5]
z_dim = h_dim[1]
regular = 1e-5
max_lr = 1e-6
base_lr = 1e-5


n_epochs = 1000

optimizer = tf.keras.optimizers.Adam(lr = base_lr)

In [6]:
def result_plot (x_train_mix, y_train_mix, ae_model, center, radius = None):
    output = ae_model(x_train_mix, training=False)
    if radius:
        trained_radius = radius.numpy()
    else:
        trained_radius = get_radius(output, nu)
    r = trained_radius**2
    dists = distance(x_train_mix, ae_model, center)
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
    ax.scatter(x_train_mix[dists>=r][:,0], x_train_mix[dists>=r][:,1],x_train_mix[dists>=r][:,2] ,s=0.9, label='anomaly')
    plt.legend()
    plt.show()
    result(dists, y_train_mix.astype(int), r )
    return trained_radius

## 利用aurtoencoder

In [7]:
def make_ae_model():

    inputs = tf.keras.Input(shape=(size,), name='inputs')

    x = tf.keras.layers.Dense(h_dim[0], activation='linear', use_bias=False)(inputs)
    x = tf.keras.layers.Dense(h_dim[1], activation='linear', use_bias=False, name = 'hidden')(x)
    x = tf.keras.layers.Dense(h_dim[0], activation='linear', use_bias=False)(x)
    x = tf.keras.layers.Dense(size, activation='linear', use_bias=False)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

ae_model = make_ae_model()
ae_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 25)                250       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


In [8]:
ae_model = make_ae_model()
ae_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_5 (Dense)              (None, 25)                250       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


## 建構模型

In [9]:
def make_model(model):
    
    model.trainable = True
    x = model.get_layer('hidden').output

    model_out = tf.keras.Model(inputs = model.input, outputs = x)
    
    return model_out

model = make_model(ae_model)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 25)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                250       
_________________________________________________________________
hidden (Dense)               (None, 5)                 50        
Total params: 300
Trainable params: 300
Non-trainable params: 0
_________________________________________________________________


## Train deep-SVDD

In [10]:
class train_SVDD():
    
    def __init__(self , objective , encoder, nu , z_dim = z_dim):
        
        self.objective = objective
        self.nu = nu
        self.z_dim = z_dim
        self.encoder = encoder
        self.optimizer = tf.keras.optimizers.Adam(lr = 1e-3, epsilon = 1e-6)
        self.center =  None #np.array(init_center_c(x_train, model = encoder, eps = 0.1))
        
        
    def init_center_c(self, data, eps = 0.1):
    
        #positive = tf.fill([self.z_dim], eps)
        #negative = tf.fill([self.z_dim], -eps)
        c = tf.zeros(shape = [self.z_dim], dtype = tf.float32, name = 'c')
        c = tf.reduce_mean( self.encoder(data, training = False), axis = 0, name = 'outputs') 
        # If c_i is too close to 0, set to +-eps. Reason: a zero unit can be trivially matched with zero weights.

        #c = tf.where(tf.logical_and(tf.less(c, 0), tf.greater(c, -eps)), negative, c)
        #c = tf.where(tf.logical_and(tf.greater(c, 0), tf.less(c, eps)), positive, c)
        
        return c
        
    def get_radius(self,batch_x):
        output = self.encoder(batch_x, training=False)
        dist = tf.reduce_sum(tf.math.square(output - self.center) , axis=1)

        return tfp.stats.percentile(tf.sqrt(dist), (1-self.nu)*100)


    def loss(self, objective, dist, R = None):

        if objective == 'soft-boundary':
            scores = dist - R**2
            avg_loss = R**2 + (1/nu) * tf.reduce_mean(tf.maximum(tf.zeros_like(scores), scores))

        if objective=='deep-SVDD':
            avg_loss = tf.reduce_mean(dist)
        return avg_loss

    @tf.function
    def train_model(self, batch_x, center, objective, R = None):
        with tf.GradientTape() as tape:
            output = self.encoder(batch_x, training=True)
            dist = tf.reduce_sum(tf.math.square(output - self.center) , axis=1)
            output_loss = self.loss(self.objective, dist, R)

        grads = tape.gradient(output_loss, self.encoder.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.encoder.trainable_variables))

        return output_loss, dist
    def train(self, train_dataset, x_train, n_epochs = 150, warm_up_n_epochs = 25):
        
        if self.objective == 'soft-boundary':
            self.R = tf.zeros(shape = [1], dtype = tf.float32, name = 'R')
        else:
            self.R = None
        loss_list = []
        pre_loss = 100
        loss_now = 10
        epoch = 0
        self.center = np.array(self.init_center_c(tf.convert_to_tensor(x_train), eps = 0.1))
        #for epoch in range(n_epochs):
        while abs(pre_loss - loss_now) > 1e-6:
            pre_loss = loss_now
            epoch_loss_avg = tf.metrics.Mean()

            for batch, (batch_x) in enumerate(train_dataset):
                loss_, dist = self.train_model(batch_x, self.center, self.objective, self.R )
                epoch_loss_avg(loss_)
                if (self.objective == 'soft-boundary') and (epoch >= warm_up_n_epochs):
                    self.R = self.get_radius(batch_x)
            #loss_list.append(loss_)
            loss_now = epoch_loss_avg.result()
            epoch +=1
        self.encoder.save_weights('oneclass_{}' .format(self.objective))
        #plt.plot(loss_list)
        #plt.show()
        
    def distance(self, data, load_model = True):
        if load_model:
            self.encoder.load_weights('oneclass_{}' .format
                                  (self.objective))
        output = self.encoder(data, training=False)
        dist = tf.reduce_sum(tf.math.square(output - self.center) , axis=1) ## dist**2
        
        return dist
    
    def decide_treshold(self, x_train):
        
        if self.objective == 'soft-boundary':
            radius_end = self.R.numpy()
        else:
            radius_end = self.get_radius(tf.convert_to_tensor(x_train)).numpy()
            
        return float(radius_end)
    def result_plot (self, dists, x_train_mix, radius ):
        r = radius**2
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
        plt.show()
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(x_train_mix[dists<=r][:,0], x_train_mix[dists<=r][:,1],x_train_mix[dists<=r][:,2] ,s=0.9, label='normal')
        ax.scatter(x_train_mix[dists>=r][:,0], x_train_mix[dists>=r][:,1],x_train_mix[dists>=r][:,2] ,s=0.9, label='anomaly')
        plt.legend()
        plt.show()
    
    def train_result(self, x_train_mix, show_img = False, load_model = True):
        
        radius_end = self.decide_treshold(x_train_mix)
        if show_img:
            dist = self.distance(x_train_mix, load_model = load_model).numpy()
            #self.result_plot (dist, x_train_mix, radius_end )

        return radius_end
    
    def result(self, dist, y_test, radius_end, show_img = False):
        
        y_true = y_test.astype(int)
        scores = dist - radius_end ** 2
        y_prob = 1/dist
        auc = roc_auc_score(y_true, y_prob)
        confusionmatrix = confusion_matrix(y_true, scores<0)
        if show_img:
            print('AUC:', auc)
            print('confusion_matrix:')
            print(confusionmatrix)

        return auc, confusionmatrix 
    
    def test_result(self, x_test_mix, y_test_mix, radius_end, show_img = False, load_model = True):
        
        dist = self.distance(x_test_mix, load_model = load_model).numpy()
        test_auc, confusionmatrix = self.result(dist, y_test_mix, radius_end, show_img = show_img)
        if show_img :
            self.result_plot (dist, x_test_mix, radius_end )

        return test_auc, confusionmatrix 


    

In [11]:
nu = 0.1
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [12]:
result = np.zeros((25, 3))

In [13]:
objective='deep-SVDD'
for i in range(25):
    ## 訓練 autoencoder
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 100 ,verbose = 0)
    ## 訓練 SVDD
    model = make_model(ae_model)
    SVDD = train_SVDD(objective, model, nu = nu, z_dim = z_dim)
    SVDD.train(train_dataset, x_train_mix)
    radius_end = SVDD.train_result(x_train_mix)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix, radius_end)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.8917977553310886
epoch =  1 auc =  0.8923272727272727
epoch =  2 auc =  0.8914381593714927
epoch =  3 auc =  0.8875933782267116
epoch =  4 auc =  0.8873414141414141
epoch =  5 auc =  0.8872951739618405
epoch =  6 auc =  0.8887519640852974
epoch =  7 auc =  0.8887196408529742
epoch =  8 auc =  0.887490684624018
epoch =  9 auc =  0.8873345679012347
epoch =  10 auc =  0.8932391694725028
epoch =  11 auc =  0.8878255892255892
epoch =  12 auc =  0.8877620089786756
epoch =  13 auc =  0.8927526374859709
epoch =  14 auc =  0.8896177328843995
epoch =  15 auc =  0.8911936026936027
epoch =  16 auc =  0.8873379349046016
epoch =  17 auc =  0.891829292929293
epoch =  18 auc =  0.8891592592592593
epoch =  19 auc =  0.8874216610549944
epoch =  20 auc =  0.8872253647586981
epoch =  21 auc =  0.88850645342312
epoch =  22 auc =  0.8896350168350168
epoch =  23 auc =  0.8941353535353534
epoch =  24 auc =  0.8872497194163861


In [14]:
np.mean(result, 0)

array([12.        ,  0.88939923, 58.49415525])

In [15]:
np.std(result, 0)

array([7.21110255e+00, 2.19416915e-03, 4.32613925e+01])

In [138]:
nu = 0.2
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [139]:
result = np.zeros((25, 3))

In [140]:
objective='deep-SVDD'
for i in range(25):
    ## 訓練 autoencoder
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 100, verbose = 0)
    ## 訓練 SVDD
    model = make_model(ae_model)
    SVDD = train_SVDD(objective , model, nu = nu, z_dim = z_dim)
    SVDD.train(train_dataset, x_train_mix)
    radius_end = SVDD.train_result(x_train_mix)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix, radius_end)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.901297932330827
epoch =  1 auc =  0.9033935463659146
epoch =  2 auc =  0.9011051378446114
epoch =  3 auc =  0.9009980576441103
epoch =  4 auc =  0.9009515037593985
epoch =  5 auc =  0.9054993107769422
epoch =  6 auc =  0.9010295112781955
epoch =  7 auc =  0.900843045112782
epoch =  8 auc =  0.9009862155388471
epoch =  9 auc =  0.9008941102756891
epoch =  10 auc =  0.9009328947368421
epoch =  11 auc =  0.9035059523809524
epoch =  12 auc =  0.9012363408521303
epoch =  13 auc =  0.9025967418546367
epoch =  14 auc =  0.901037656641604
epoch =  15 auc =  0.9034134085213033
epoch =  16 auc =  0.905084335839599
epoch =  17 auc =  0.9028522556390978
epoch =  18 auc =  0.9013080827067669
epoch =  19 auc =  0.9012379699248121
epoch =  20 auc =  0.9009692982456141
epoch =  21 auc =  0.901091290726817
epoch =  22 auc =  0.9010911967418546
epoch =  23 auc =  0.9010055764411028
epoch =  24 auc =  0.902831015037594


In [141]:
np.mean(result, 0)

array([12.        ,  0.9018877 , 84.59634295])

In [142]:
np.std(result, 0)

array([7.21110255e+00, 1.33808815e-03, 5.46605662e+01])

In [143]:
nu = 0.3
x_train_mix, y_train_mix = gernate_data(6000, nu, 16, range_ = 10, multi = 15, size = size)
x_test_mix, y_test_mix = gernate_data(10000, nu, 16,range_ = 10, multi = 15, size = size)
n_samples = len(x_train_mix)


batch_size = 2000
train_buf = x_train_mix.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_mix)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [144]:
result = np.zeros((25, 3))

In [145]:
objective='deep-SVDD'
for i in range(25):
    ## 訓練 autoencoder
    pre_time = time.time()
    ae_model = make_ae_model()
    ae_model.compile(optimizer="Adam", loss="mse")
    ae_model.fit(x_train_mix, x_train_mix, epochs = 100, verbose = 0)
    ## 訓練 SVDD
    model = make_model(ae_model)
    SVDD = train_SVDD(objective , model, nu = nu, z_dim = z_dim)
    SVDD.train(train_dataset, x_train_mix)
    radius_end = SVDD.train_result(x_train_mix)
    auc , matrix = SVDD.test_result(x_test_mix, y_test_mix, radius_end)
    train_time = time.time() - pre_time
    print('epoch = ', i, 'auc = ', auc)
    result[i,0] = i
    result[i,1] = auc
    result[i,2] = train_time

epoch =  0 auc =  0.8695532857142857
epoch =  1 auc =  0.8686713333333334
epoch =  2 auc =  0.8685629047619048
epoch =  3 auc =  0.8707925714285715
epoch =  4 auc =  0.8682555238095239
epoch =  5 auc =  0.8687573333333333
epoch =  6 auc =  0.8689124285714287
epoch =  7 auc =  0.8689834761904763
epoch =  8 auc =  0.8689150476190477
epoch =  9 auc =  0.8722992857142855
epoch =  10 auc =  0.8688273333333333
epoch =  11 auc =  0.8690736190476192
epoch =  12 auc =  0.8681613809523809
epoch =  13 auc =  0.869062523809524
epoch =  14 auc =  0.868827857142857
epoch =  15 auc =  0.8693151904761907
epoch =  16 auc =  0.8687817619047619
epoch =  17 auc =  0.8693990476190476
epoch =  18 auc =  0.8686228571428573
epoch =  19 auc =  0.8686087142857142
epoch =  20 auc =  0.8686744285714285
epoch =  21 auc =  0.8723288095238095
epoch =  22 auc =  0.8682281428571429
epoch =  23 auc =  0.8686672380952382
epoch =  24 auc =  0.8679190238095238


In [146]:
np.mean(result, 0)

array([12.        ,  0.86912804, 81.01051138])

In [147]:
np.std(result, 0)

array([7.21110255e+00, 1.08365651e-03, 5.12258896e+01])